# How to Render a Metal Design into Ansys

## Prerequisite
You need to have Ansys installed locally (any version) - at the time of this notebook, Ansys is only supported in Windows.

## 1. Perform the necessary imports and create a design in Metal

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [ ]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer
QAnsysRenderer.default_options

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

In [ ]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this notebook, the design consists of 4 qubits and 4 CPWs

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '5.7 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Render into Ansys HFSS
#### 2.1 Create a Metal renderer for Ansys HFSS

In [ ]:
fourq_hfss = design.renderers.hfss

#### 2.2 Setup an Ansys project
To setup the project **manually**, follow these instructions:
1. Launch `ANSYS Electronics Desktop yyyy Rx` (from your Windows Start menu).
2. Create a new Ansys project by clicking on the `New` icon at the top left. (or open an existing project)

Alternatively, you can **automatically** set up the project by executing the following two cells. Make sure to wait after executing the first cell for Ansys to completely open. Only then execute the second cell.

Note about Ansys version: open_ansys() will look by default for the 2020 R2 version of Ansys. You can easily reroute it to your Ansys of choice by providing the name of the environment variable that contains the path (path_env) or the path itself (path)

In [ ]:
fourq_hfss.open_ansys()   # this opens Ansys 2020 R2 if present
# fourq_hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# fourq_hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM20.2\Win64')
# fourq_hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM20.2/Win64')

NOTE: A new project should have automatically opened with the execution of the cell above. If not, uncomment the cell below and execute it. You can also load an existing project by passing the project information to the `connect_ansys()`.

In [ ]:
# fourq_hfss.new_ansys_project()

#### 2.3 Connect the Metal renderer with the Ansys project

In [ ]:
# Open either a new or existing design based on default options
fourq_hfss.connect_ansys()
# hfss.connect_ansys('C:\\project_path\\', 'Project1')  # Example of opening a saved project

You can optionally indicate with the optional parameters whether you intend to open and use a previously saved project. <br>
Make sure that the saved project contains at least one design, or the method will produce an error

#### 2.4 Setup an Ansys HFSS design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert HFSS Design`.
3. Change the HFSS design to eigenmode by right-clicking on the HFSSdesign1 that just got created inside your project (left panel) and then selecting: `Solution Type...`. Select Modal.

To **create** a new design **automatically**, execute the following cell<br>
Note: If a design named `HFSSTransmonQubit` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `HFSSTransmonQubit1`, `HFSSTransmonQubit2`, etc.

In [ ]:
# The project that was active when fourq_hfss.connect_ansys() happened, is where the new design will be inserted.
# Please observe in Ansys, after the hfss eigenmode design is inserted, and if the name has been used before
# an integer is added to the suffix of the name.
# Every time you add design with the same name, the integer is increased.
fourq_hfss.add_eigenmode_design("HFSSMetalEigenmode")

To **create** a new design for driven modal **automatically**, execute the following cell

In [ ]:
# The project that was active when fourq_hfss.connect_ansys() happened, is where the new design will be inserted.
# Please observe in Ansys, after the hfss driven modal design is inserted, and if the name has been used before
# an integer is added to the suffix of the name.
# Every time you add design with the same name, the integer is increased.
fourq_hfss.add_driven_modal_design("HFSSMetalDrivenModal")

To **select** and existing design, you need to be aware of the following constraints:
* Metal will only communicate with the active design
* Metal needs the design to be empty

Therefore, if you would like to use an existing design, you need to manually activate it from the Ansys GUI and empty it:
1. To activate the intended Ansys HFSS design, simply double click on its name in the leftmost panel in the Ansys GUI. After this, you might need to re-run the connect_ansys() *without parameters*.
2. Empty the Ansys HFSS design by executing the following cell. This is equivalent to doing the following from the Ansys GUI: hold down Ctrl + A (to select everything in the design) -> right clicking -> Edit -> Delete.

In [ ]:
obj_names = fourq_hfss.pinfo.get_all_object_names()
if obj_names:
    fourq_hfss.clean_active_design()

#### 2.5 Render some component from the Metal design

There are 4 different render examples below. You need to render only one of them. <br>
If you intend to render a different example or something else, please first clear the design by re-executing the cell above.

In [ ]:
#    Render the whole Metal design into Ansys HFSS.
#    To denote which boudning box the user wants.   Argument  box_plus_buffer = True by default. 
fourq_hfss.render_design([], [])

In [ ]:
#    Render the whole Metal design into Ansys HFSS.
#    The  argument  box_plus_buffer = True by default
#    If switch to False, the chip size, which comes from DesignPlanar will be used.

#    The chip size is larger than the 
fourq_hfss.render_design([], [], box_plus_buffer=False)

obj_names = fourq_hfss.pinfo.get_all_object_names()
if obj_names:
    fourq_hfss.clean_active_design()

#   Change the chip size to see what happens when the geometry is larger than the chip size.  
#   You will get a warning, but will still render to Ansys.
design._chips['main']['size']['size_x'] = '4mm'
fourq_hfss.render_design([], [], box_plus_buffer=False)


In [ ]:
#    Render a single qubit with 2 endcaps into Ansys HFSS.
# fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])

In [ ]:
#    Render 2 qubits and 2 endcaps, one per qubit, into Ansys HFSS.
# fourq_hfss.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])

In [ ]:
#    Render a single qubit with 1 open endcap and 1 lumped port with 70 Ohm impedance.
# fourq_hfss.render_design(['Q2'], [('Q2', 'a')], [('Q2', 'b', '70')])

The last cell above has a 70 Ohm termination intended for Driven-Modal solution. This will require to change the solution type in Ansys by doing this: Right click on the Ansys HFSS design in the leftmost panel of the Ansys GUI, then select "Solution Type...", and choose Modal.

**Finally** Disconnect the Metal renderer from the Ansys session. You will not be able to close Ansys without executing this.

In [ ]:
fourq_hfss.disconnect_ansys()

## 3. Render into Ansys Q3D
#### 3.1 Create a Metal renderer for Ansys Q3D

In [ ]:
fourq_q3d = design.renderers.q3d

#### 3.2 Setup an Ansys project
Skip this session if you have already executed cells in 2.2, and you left Ansys open. Indeed, it does not matter which renderer has opened Ansys or created the project. It will suffice to connect the fourq_q3d renderer to the Ansys session (section 3.3).

The following cell will open a new session of Ansys with a brand new project

In [ ]:
fourq_q3d.open_ansys()   # this opens Ansys 2020 R2 if present

#### 3.3 Connect the Metal renderer with the Ansys project

In [ ]:
# Open either a new or existing design based on default options
fourq_q3d.connect_ansys()

You can optionally indicate with the method in the above cell whether you intend to open and use a previously saved project.
Make sure that the saved project contains at least one design, or the method will produce an error

#### 3.4 Setup an Ansys Q3D design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert Q3D Extractor Design`.

To **create** a new Q3D design **automatically**, execute the following cell.<br>
Note: If a design named `Q3dMetalDesign` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `Q3dMetalDesign1`, `Q3dMetalDesign2`, etc.

In [ ]:
fourq_q3d.add_q3d_design("Q3dMetalDesign")

To **select** and existing design, you need to be aware of the following constraints:
* Metal will only communicate with the active design
* Metal needs the design to be empty

Therefore, if you would like to use an existing design, you need to manually activate it from the Ansys GUI and empty it:
1. To activate the intended Ansys Q3D design, simply double click on its name in the leftmost panel in the Ansys GUI. After this, you might need to re-run the connect_ansys() *without parameters*.
2. Empty the Ansys Q3D design by executing the following cell. This is equivalent to doing the following from the Ansys GUI: hold down Ctrl + A (to select everything in the design) -> right clicking -> Edit -> Delete.

In [ ]:
if fourq_q3d.pinfo is not None:
    obj_names = fourq_q3d.pinfo.get_all_object_names()
    if obj_names:
        fourq_q3d.clean_active_design()

#### 3.5 Render some component from the Metal design

There are 3 different render examples below. You need to render only one of them. <br>
If you intend to render a different example or something else, please first clear the design by re-executing the cell above.

In [ ]:
#     Render the whole Metal design into Ansys Q3D.
fourq_q3d.render_design([], [])

In [ ]:
#     Render a single qubit with 2 endcaps into Ansys Q3D.
# fourq_q3d.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])

In [ ]:
#     Render 2 qubits and 2 endcaps, one per qubit, into Ansys Q3D.
# fourq_q3d.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])

**Finally** Disconnect the Metal renderer from the Ansys session. You will not be able to close Ansys without executing this.

In [ ]:
fourq_q3d.disconnect_ansys()

## References - Miscellaneous pyEPR/Ansys commands
The following commands are for reference only to better understand how the backend code works. They're not meant to be run directly in this notebook as part of the guide.

import pyEPR as epr

Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

Access methods within HfssDesign class in pyEPR:

epr.ansys.HfssDesign.create_dm_setup
epr.ansys.HfssDesign.create_q3d_setup

Get project and design names:

pinfo.project_name
design._design.GetName()

Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')